In [1]:
# -*- coding: utf-8 -*-
%matplotlib inline
import numpy as np
import dynamical_barrier as db
from lmfit import Model
import os
def exponential(x,amp,decay_rate,yoffset,xoffset):
    return amp*np.exp(decay_rate*(x-xoffset))+yoffset
from scipy.interpolate import griddata
from matplotlib import pyplot as pl
import multiprocessing
from multiprocessing import Pool
from tqdm import notebook

if not os.path.exists('ETD_Temp'):
    os.mkdir('ETD_Temp')
# folder = os.getcwd()+'//ETD_Temp'

# #https://stackoverflow.com/questions/185936/how-to-delete-the-contents-of-a-folder-in-python
# for the_file in os.listdir(folder):
#     file_path = os.path.join(folder, the_file)
#     try:
#         if os.path.isfile(file_path):
#             os.unlink(file_path)
#         #elif os.path.isdir(file_path): shutil.rmtree(file_path)
#     except Exception as e:
#         print(e)
a=3.
b=3.5
a1=3.5033
b1=1
c=a1-2.1
k = np.sqrt(np.absolute(a**2-b**2))
q1=400
q2=10
h1=0.2
h2=0.8/40.
Nmax=500
Dx_start=0.5
yx_start=0.2
focdist=0.1
cut_angle=np.pi/2
#allow them to cross below threshold
label_threshold=10
# iteration
def Pool1(m):
    Dx=Dx_start
    yx=yx_start+h2*m
#     cut_angle=np.pi/2
#     focdist=0.55
    Penrose=db.geometry(a,b,a1,b1,c,focdist,cut_angle)
    with open(os.getcwd()+"//ETD_Temp//green_focdist=%.2f,cut_angle=%.1f" % (focdist,cut_angle*180/np.pi)+","+"ETD_%d.txt" % m,"w") as fp:
        count=int(q1*q2)
        label=np.ones(q1*q2)*label_threshold
        time=0
        DB=np.zeros(q1*q2)
        test=Penrose.IC_generator(Dx,yx,h1,h2,q1,q2)
        print(m,'Working')
        for i in range (0,q1*q2,1):
            init_ang_diff=Penrose.nangle_pt(test[i][0],test[i][1],test[i][3])-test[i][2]
            DB[i]=Penrose.Dynamical_Barrier(test[i][0],test[i][1],init_ang_diff,test[i][3])
        for j in notebook.trange(Nmax):
            if(j==0):
                irayset = np.array(Penrose.IC_generator(Dx,yx,h1,h2,q1,q2))
            else:
                time=time+1
                for i in range(0,q1*q2,1):
                    fray=Penrose.reflected_ray(irayset[i])
                    ang_diff=Penrose.nangle_pt(fray[0],fray[1],fray[3])-fray[2]
                    data=[Penrose.eta(fray[0],fray[1],fray[3])/Penrose.eta(0,0,14),np.sin(ang_diff)]
                    #Dynamical barrier count
                    #start black or red,
                    if(((DB[i]==1 or DB[i]==4) and (Penrose.Dynamical_Barrier(fray[0],fray[1],ang_diff,fray[3])!=DB[i]) and label[i]!=0 and not ((DB[i]==1 and Penrose.Dynamical_Barrier(fray[0],fray[1],ang_diff,fray[3])==4) or (DB[i]==4 and Penrose.Dynamical_Barrier(fray[0],fray[1],ang_diff,fray[3])==1))) or (DB[i]==3 and j==Nmax-1 and label[i]!=0)):
                        label[i]=label[i]-1
                        if(label[i]==0):
                            count=count-1
                            test_ang_diff=Penrose.nangle_pt(test[i][0],test[i][1],test[i][3])-test[i][2]
                            test_data=[Penrose.eta(test[i][0],test[i][1],test[i][3])/Penrose.eta(0,0,14),np.sin(test_ang_diff)]
                            fp.write('%f %f %d\n' % (test_data[0],test_data[1],time))
                    if(j==Nmax-1 and (DB[i]!=1 and DB[i]!=4)):
                        test_ang_diff=Penrose.nangle_pt(test[i][0],test[i][1],test[i][3])-test[i][2]
                        test_data=[Penrose.eta(test[i][0],test[i][1],test[i][3])/Penrose.eta(0,0,14),np.sin(test_ang_diff)]
                        fp.write('%f %f %d\n' % (test_data[0],test_data[1],0))
                    for m in range(4):
                        irayset[i][m]=fray[m]
    # cut_angle=np.pi/2.
thread_num=40
# focdistlist=[0.05*n for n in range(1,7)]
num_list=[x for x in range(0,thread_num,1)]
with Pool(thread_num) as p:
    p.map(Pool1,num_list)

x=[]
y=[]
t=[]
for m in np.arange(thread_num):
    data=open(os.getcwd()+"//ETD_Temp//green_focdist=%.2f,cut_angle=%.1f" % (focdist,cut_angle*180/np.pi)+",ETD_%d.txt" % m).read().split()
    data=list(map(float,data))
    x=x+[data[i] for i in range(0,len(data),3)]
    y=y+[data[i] for i in range(1,len(data),3)]
    t=t+[int(data[i]) for i in range(2,len(data),3)]
x=np.array(x)
y=np.array(y)
t=np.array(t)
xi=np.linspace(Dx_start,Dx_start+h1,1000)
yi=np.linspace(yx_start,yx_start+h2*thread_num,1000)
zi=griddata((x,y),t,(xi[None,:],yi[:,None]),method='cubic')
pl.contourf(xi,yi,zi,levels=100,vmin=0,vmax=np.max(np.nan_to_num(zi)),cmap='jet')
pl.title('ETD of cut width=%.2f, cut angle=%.1f'u'\N{DEGREE SIGN}' % (focdist,cut_angle*180/np.pi))
pl.xlim(Dx_start,Dx_start+h1*(q1-1)/q1)
pl.ylim(yx_start,yx_start+h2*(thread_num-1))
# pl.xlim(0,1)
# pl.ylim(-1,1)
pl.xlabel(r'$\eta$')
pl.ylabel(r'$\sin(\chi)$')
pl.colorbar()
pl.savefig(os.getcwd()+'//ETD_Temp//green_focdist=%.2f,cut_angle=%.1f' % (focdist,cut_angle*180/np.pi)+',ETD.png',dpi=600)

19 Working
9 Working
29 Working
6 Working
1 Working
4 Working
13 Working
25 Working
35 Working
24 Working
37 Working
27 Working
20 Working7
 Working
21 Working
30 Working
32 Working
0 Working
39 Working
31 Working
8 Working
15 Working
2 Working
16 Working
34 Working
11 Working
12 Working
5 Working
18 Working
26 Working
14 Working
38 Working
10 Working
3 Working
33 Working
36 Working
22 Working
28 Working
23 Working
17 Working


KeyboardInterrupt: 

In [ ]:
print(x[np.argmax(t)])
print(y[np.argmax(t)])
print(np.max(t))
print(np.max(np.nan_to_num(zi)))
print(yx_start+h2*thread_num)
print(yx_start)
print(h2)
print(np.max(y))
print(np.max(x),Dx_start+h1)